在搭建好网络模型之后，需要对网络模型中的权值进行初始化。适当的权值初始化可以加快模型的收敛，而不恰当的权值初始化可能引发梯度消失或者梯度爆炸，最终导致模型无法训练。

# 1. 梯度消失与梯度爆炸

![](图1.png)

根据链式求导法则推出的结果可以看到，${W}_{2}$ 的梯度依赖于前一层的输出 $H_{1}$，因此：

- 如果$H_{1}$ 趋近于零，那么 ${W}_{2}$ 的梯度也接近于 0，造成梯度消失；

- 如果$H_{1}$ 趋近于无穷大，那么 ${W}_{2}$ 的梯度也接近于无穷大，造成梯度爆炸；

**要避免梯度爆炸或者梯度消失，就要严格控制网络层输出数值的范围，各个网络层输出数值的范围保持在一个固定的范围内，我们可以使用方差/标准差来描述数值的变化范围，通常使各个网络层输出数值的方差在 1 附近（0 均值）。**

# 2. 仅包含全连接层时，全连接层权值的初始化：

In [1]:
import torch
import torch.nn as nn

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data)  # 权值初始化为标准正态分布，mean=0, std=1

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 100
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:15.959932327270508
layer:1, std:256.6237487792969
layer:2, std:4107.24560546875
layer:3, std:65576.8125
layer:4, std:1045011.875
layer:5, std:17110408.0
layer:6, std:275461408.0
layer:7, std:4402537984.0
layer:8, std:71323615232.0
layer:9, std:1148104736768.0
layer:10, std:17911758454784.0
layer:11, std:283574846619648.0
layer:12, std:4480599809064960.0
layer:13, std:7.196814275405414e+16
layer:14, std:1.1507761512626258e+18
layer:15, std:1.853110740188555e+19
layer:16, std:2.9677725826641455e+20
layer:17, std:4.780376223769898e+21
layer:18, std:7.613223480799065e+22
layer:19, std:1.2092652108825478e+24
layer:20, std:1.923257075956356e+25
layer:21, std:3.134467063655912e+26
layer:22, std:5.014437766285408e+27
layer:23, std:8.066615144249704e+28
layer:24, std:1.2392661553516338e+30
layer:25, std:1.9455688099759845e+31
layer:26, std:3.0238180658999113e+32
layer:27, std:4.950357571077011e+33
layer:28, std:8.150925520353362e+34
layer:29, std:1.322983152787379e+36
layer:30, std

上面这个网络模型运行前向传播之后，我们可以看到每一层的标准差越来越大，并在在 30 层时超出了数据可以表示的范围。

<br/>

下面推导为什么网络层输出的标准差越来越大：

**3 个基础公式：**

$E(X \times Y)=E(X) \times E(Y)$：两个相互独立的随机变量的乘积的期望等于它们的期望的乘积。

$D(X)=E(X^{2}) - [E(X)]^{2}$：一个随机变量的方差等于它的平方的期望减去期望的平方

$D(X+Y)=D(X)+D(Y)$：两个相互独立的随机变量之和的方差等于它们的方差的和。

<br/>

**根据上述 3 个基础公式，可以推导出两个相互独立的随机变量的乘积的方差如下：**

$D(X \times Y)=E[(XY)^{2}] - [E(XY)]^{2}=D(X) \times D(Y) + D(X) \times [E(Y)]^{2} + D(Y) \times [E(X)]^{2}$

如果$E(X)=0$，$E(Y)=0$，那么$D(X \times Y)=D(X) \times D(Y)$

<br/>

**将该推导结果应用到神经元节点上：**

![](图2.png)

我们以第一个隐藏层的第一个神经元 $\mathrm{H}_{11}$ 为例，X 是输入层的输入值，W 是由输入层的输入值的权值，n 是输入层神经元的数量，输入值 X 和权值 W 都是服从 $N(0,1)$ 的正态分布，所以第一个隐藏层的第一个神经元的输出值的方差为：$\begin{aligned} \mathbf{D}\left(\mathrm{H}_{11}\right) &=n \end{aligned}$，标准差为：$\operatorname{std}\left(\mathrm{H}_{11}\right)=\sqrt{n}$。

进一步可知，第一个隐藏层的输出值的方差也为 n，标准差也为 $\sqrt{n}$。

输入层的输入值的方差为 1，所以由输入层的输入值到第一个隐藏层的输出值，方差扩大了 n 倍，标准差扩大了 $\sqrt{n}$ 倍。

由不严谨但正确的类推可知，**若权值 W 的标准差都是 1，则每经过一个网络层，网络层的输出值的方差就会扩大 n 倍，标准差就会扩大$\sqrt{n}$倍，n 为每层神经元个数**。此结论也可以由上面代码输出的每个网络层的输出值得到验证，每层神经元的数量为 256，网络层的输出值的标准差逐层扩大约 16 倍。

如果每经过一个网络层，网络层的输出值的方差就会扩大 n 倍，这样经过数十个网络层之后，网络层的输出值的方差就会大到计算机无法表示，因此我们需要采取措施避免这种情况发生。**本网络层输出值的方差与上一网络层的神经元个数、上一网络层的输出值（亦即本网络层的输入值）的方差、上一网络层输出值的权值的方差有关**，这其中比较好改变的是上一网络层输出值（亦即本网络层的输入值）的权值的方差 $D(W)$，**使 $D(W)= \frac{1}{n}$，$std(W)=\sqrt\frac{1}{n}$，可以确保每经过一个网络层，网络层输出值的方差大致不变**。

<br/>

下面通过代码展示：将权值 W 的标准差设置为 $\sqrt\frac{1}{n}$ 对前向传播过程中每个网络层输出值的影响

In [2]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_nums))  # 权值初始化为正态分布，mean=0, std=根号1/n

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:1.0022112131118774
layer:1, std:1.0033732652664185
layer:2, std:1.0147582292556763
layer:3, std:1.0430957078933716
layer:4, std:1.0486743450164795
layer:5, std:1.0437393188476562
layer:6, std:1.0411100387573242
layer:7, std:1.0687036514282227
layer:8, std:1.0842468738555908
layer:9, std:1.0959652662277222
layer:10, std:1.0800222158432007
layer:11, std:1.0889637470245361
layer:12, std:1.099170207977295
layer:13, std:1.0889158248901367
layer:14, std:1.0736167430877686
layer:15, std:1.0679516792297363
layer:16, std:1.0509815216064453
layer:17, std:1.0493961572647095
layer:18, std:1.0645091533660889
layer:19, std:1.0877567529678345
tensor([[-0.9184, -0.9523, -0.1050,  ...,  0.5743, -0.8024,  1.1647],
        [ 0.1230,  3.1904, -1.1976,  ..., -1.4894,  0.6069,  0.3317],
        [-1.2829,  0.9495, -0.0994,  ..., -0.8504,  0.0426, -0.5808],
        ...,
        [ 0.4939,  1.2548, -0.6355,  ..., -0.9356,  0.6341, -0.2524],
        [ 0.0402,  0.6965, -0.5138,  ..., -0.3765,  0.5294

从上述代码的运行结果可以看到，通过将权值 W 的标准差设置为 $\sqrt\frac{1}{n}$，前向传播过程中每个网络层输出值的标准差都保持在 1 左右。

<br/>

上述前向传播是模型仅包含全连接层（线性变换），不包含激活函数层（非线性变换）时的实验结果，如果在 forward() 中添加激活函数层，每一层的输出值的方差会越来越小，会导致梯度消失：

In [3]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.tanh(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_nums))  # 权值初始化为正态分布，mean=0, std=根号1/n

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.627248227596283
layer:1, std:0.48665833473205566
layer:2, std:0.41344448924064636
layer:3, std:0.3640071451663971
layer:4, std:0.32617107033729553
layer:5, std:0.2971877455711365
layer:6, std:0.27194929122924805
layer:7, std:0.2612219452857971
layer:8, std:0.2483828216791153
layer:9, std:0.23589369654655457
layer:10, std:0.22312353551387787
layer:11, std:0.21378853917121887
layer:12, std:0.20496906340122223
layer:13, std:0.19552716612815857
layer:14, std:0.1851242482662201
layer:15, std:0.1773529201745987
layer:16, std:0.17073562741279602
layer:17, std:0.16620661318302155
layer:18, std:0.16274142265319824
layer:19, std:0.1617128998041153
tensor([[-0.1143, -0.1310, -0.0734,  ...,  0.1242, -0.1479,  0.1477],
        [ 0.0461,  0.3603, -0.1624,  ..., -0.2285,  0.0689, -0.0195],
        [-0.0896,  0.0675,  0.0440,  ..., -0.1235, -0.0068, -0.0136],
        ...,
        [ 0.0708,  0.2205, -0.0834,  ..., -0.1684,  0.1373,  0.0136],
        [ 0.0098,  0.0694, -0.0805,  ..., -0.0

通过上述代码输出的每一网络层的输出值可知，在添加激活函数层之后，每一层输出值的标准差越来越小，随着网络层加深，可能会导致梯度消失。

为了解决这个问题，出现了 Xavier 初始化方法（针对饱和激活函数，如 sigmoid 和 tanh）与 Kaiming 初始化方法（针对非饱和激活函数，如 relu）。

# 2. 包含全连接层和饱和激活函数层时，全连接层权值的初始化：Xavier 权值初始化方法

目标：保持网络层输出值的方差在恰当范围，通常在 1 左右

适用对象：主要针对饱和激活函数，如 sigmoid 和 tanh

<br/>

若保持网络层输出值的方差在恰当范围，且同时考虑前向传播和反向传播，需要满足两个等式：

$\boldsymbol{n}_{\boldsymbol{i}} * \boldsymbol{D}(\boldsymbol{W})=\mathbf{1}$ 

$\boldsymbol{n}_{\boldsymbol{i+1}} * \boldsymbol{D}(\boldsymbol{W})=\mathbf{1}$

可得：$D(W)=\frac{2}{n_{i}+n_{i+1}}$。

<br/>

若使 Xavier 方法初始化的权值服从均匀分布 $U[-a, a]$，那么方差 $D(W)=\frac{(-a-a)^{2}}{12}=\frac{(2 a)^{2}}{12}=\frac{a^{2}}{3}$;

令$\frac{2}{n_{i}+n_{i+1}}=\frac{a^{2}}{3}$，解得：$\boldsymbol{a}=\frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}$；

**所以 Xavier 方法将权值  $W$  初始化为均匀分布 $U\left[-\frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}, \frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}\right]$ 时，可以保持网络层输出值的方差在恰当范围**（同时考虑了前向传播和反向传播）。

代码实验：

In [4]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.tanh(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                a = np.sqrt(6 / (self.neural_nums + self.neural_nums))
                tanh_gain = nn.init.calculate_gain('tanh')
                a *= tanh_gain  # 上下限乘以 tanh 激活函数对输入值标准差的缩小比例
                nn.init.uniform_(m.weight.data, -a, a)  # 权值初始化为 [-a, a] 的均匀分布

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.7564043998718262
layer:1, std:0.6912741661071777
layer:2, std:0.6716636419296265
layer:3, std:0.6640926599502563
layer:4, std:0.650257408618927
layer:5, std:0.6443925499916077
layer:6, std:0.6554051041603088
layer:7, std:0.6477739214897156
layer:8, std:0.6523249745368958
layer:9, std:0.6515239477157593
layer:10, std:0.6521815061569214
layer:11, std:0.6556642055511475
layer:12, std:0.6526561379432678
layer:13, std:0.6564072370529175
layer:14, std:0.6549282670021057
layer:15, std:0.6534885168075562
layer:16, std:0.6481824517250061
layer:17, std:0.6542978882789612
layer:18, std:0.656050443649292
layer:19, std:0.6504389047622681
tensor([[ 0.9328,  0.9638,  0.2703,  ..., -0.3082,  0.4021,  0.8018],
        [-0.5733,  0.6387,  0.7284,  ...,  0.4264,  0.7715,  0.0212],
        [-0.2264,  0.5689, -0.8231,  ..., -0.9744,  0.6164, -0.1837],
        ...,
        [-0.9853, -0.7947, -0.1297,  ...,  0.4011,  0.6222, -0.8152],
        [-0.9399, -0.4636, -0.2226,  ...,  0.3696, -0.4646,

在添加了饱和激活函数层之后，通过将权值初始化为 `[-a, a]` 的均匀分布，以及上下限乘以 tanh 激活函数对输入值标准差的缩小比例，我们可以看到，前向传播过程每个网络层的输出值的标准差保持在 0.65 左右，解决了每一层输出值的标准差越来越小的问题。

<br/>

PyTorch 也提供了 Xavier 初始化方法，可以直接调用：

In [5]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.tanh(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                tanh_gain = nn.init.calculate_gain('tanh')
                nn.init.xavier_uniform_(m.weight.data, gain=tanh_gain)  # 直接调用 PyTorch 提供的 Xavier 初始化方法

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.7564043998718262
layer:1, std:0.6912741661071777
layer:2, std:0.6716636419296265
layer:3, std:0.6640926599502563
layer:4, std:0.650257408618927
layer:5, std:0.6443925499916077
layer:6, std:0.6554051041603088
layer:7, std:0.6477739214897156
layer:8, std:0.6523249745368958
layer:9, std:0.6515239477157593
layer:10, std:0.6521815061569214
layer:11, std:0.6556642055511475
layer:12, std:0.6526561379432678
layer:13, std:0.6564072370529175
layer:14, std:0.6549282670021057
layer:15, std:0.6534885168075562
layer:16, std:0.6481824517250061
layer:17, std:0.6542978882789612
layer:18, std:0.656050443649292
layer:19, std:0.6504389047622681
tensor([[ 0.9328,  0.9638,  0.2703,  ..., -0.3082,  0.4021,  0.8018],
        [-0.5733,  0.6387,  0.7284,  ...,  0.4264,  0.7715,  0.0212],
        [-0.2264,  0.5689, -0.8231,  ..., -0.9744,  0.6164, -0.1837],
        ...,
        [-0.9853, -0.7947, -0.1297,  ...,  0.4011,  0.6222, -0.8152],
        [-0.9399, -0.4636, -0.2226,  ...,  0.3696, -0.4646,

# 3. 包含全连接层和非饱和激活函数层时，全连接层权值的初始化：Kaiming 权值初始化方法

若激活函数是非饱和激活函数（如 Relu），Xavier 方法将不再适用，2015 年针对 ReLU 及其变种等激活函数提出了 Kaiming 初始化方法。

目标：保持网络层输出值的方差在恰当范围，通常在 1 左右

适用对象：Relu 及其变种

<br/>

若保持网络层的输出值在一个适当的范围，需要满足：（证明略）

$\mathrm{D}(W)=\frac{2}{n_{i}}$（针对 ReLU）

$\mathrm{D}(W)=\frac{2}{\left(1+a^{2}\right) n_{i}}$，$\operatorname{std}(W)=\sqrt{\frac{2}{\left(1+a^{2}\right) * n_{i}}}$（针对 ReLu 的变种，a 表示负半轴的斜率，$n_{i}$ 为输入层/上一层神经元的数量）

**所以 Kaiming 方法将权值  $W$  初始化为正态分布 $N(0, std(w))$ 时，可以保持网络层输出值的方差在恰当范围。**

代码实验：

In [6]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.normal_(m.weight.data, std=np.sqrt(2 / self.neural_nums))  # 使用 Kaiming 方法初始化权值

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.8343690633773804
layer:1, std:0.8857383728027344
layer:2, std:0.8768216371536255
layer:3, std:0.8620700836181641
layer:4, std:0.8756188750267029
layer:5, std:0.8784897923469543
layer:6, std:0.8935869336128235
layer:7, std:0.8712248206138611
layer:8, std:0.9630030989646912
layer:9, std:0.9609673619270325
layer:10, std:0.8539234399795532
layer:11, std:0.8073139190673828
layer:12, std:0.9279561638832092
layer:13, std:0.9541431069374084
layer:14, std:0.9491678476333618
layer:15, std:1.0219509601593018
layer:16, std:0.8949427604675293
layer:17, std:0.916878342628479
layer:18, std:0.8786735534667969
layer:19, std:0.9656010270118713
tensor([[0.0000, 0.1963, 1.8828,  ..., 0.0000, 0.0000, 1.6750],
        [0.0000, 0.0000, 1.3597,  ..., 0.0000, 0.0000, 1.2228],
        [0.0000, 0.0000, 2.4619,  ..., 0.0000, 0.0000, 2.1729],
        ...,
        [0.0000, 0.3190, 0.8542,  ..., 0.0000, 0.0000, 1.6953],
        [0.0000, 0.0305, 1.7972,  ..., 0.0000, 0.0000, 1.9260],
        [0.0000, 0

在添加了非饱和激活函数层 Relu 之后，**通过将权值初始化为标准差为 $\sqrt{\frac{2}{n_{i}}}$ 的正态分布，我们可以看到，前向传播过程每个网络层的输出值的标准差保持在一个恰当的范围。**

<br/>

PyTorch 也提供了 Kaiming 初始化方法，可以直接调用：

In [7]:
import torch
import torch.nn as nn
import numpy as np

# 创建网络模型
class MLP(nn.Module):
    def __init__(self, neural_nums, layer_nums):
        super().__init__()
        self.linears = nn.ModuleList([nn.Linear(neural_nums, neural_nums, bias=False) for i in range(layer_nums)])   # 创建数个完全一样的全连接层
        self.neural_nums = neural_nums

    # 前向传播
    def forward(self, x):
        for (i, linear) in enumerate(self.linears):
            x = linear(x)
            x = torch.relu(x)
            print("layer:{}, std:{}".format(i, x.std()))
            if torch.isnan(x.std()):
                print("output is nan in {} layers".format(i))
                break     
        return x

    # 权值初始化
    def initialize(self):
        for m in self.modules():
            if isinstance(m, nn.Linear):
                nn.init.kaiming_normal_(m.weight.data)  # 直接调用 PyTorch 提供的 Kaiming 初始化方法

# 设置随机种子
torch.manual_seed(1)

# 实例化网络模型，运行前向传播
layer_nums = 20
neural_nums = 256
batch_size = 16

net = MLP(neural_nums, layer_nums)
net.initialize()
inputs = torch.randn((batch_size, neural_nums))  # 输入值初始化为标准正态分布，mean=0, std=1
output = net(inputs)
print(output)

layer:0, std:0.8343690633773804
layer:1, std:0.8857383728027344
layer:2, std:0.8768216371536255
layer:3, std:0.8620700836181641
layer:4, std:0.8756188750267029
layer:5, std:0.8784897923469543
layer:6, std:0.8935869336128235
layer:7, std:0.8712248206138611
layer:8, std:0.9630030989646912
layer:9, std:0.9609673619270325
layer:10, std:0.8539234399795532
layer:11, std:0.8073139190673828
layer:12, std:0.9279561638832092
layer:13, std:0.9541431069374084
layer:14, std:0.9491678476333618
layer:15, std:1.0219509601593018
layer:16, std:0.8949427604675293
layer:17, std:0.916878342628479
layer:18, std:0.8786735534667969
layer:19, std:0.9656010270118713
tensor([[0.0000, 0.1963, 1.8828,  ..., 0.0000, 0.0000, 1.6750],
        [0.0000, 0.0000, 1.3597,  ..., 0.0000, 0.0000, 1.2228],
        [0.0000, 0.0000, 2.4619,  ..., 0.0000, 0.0000, 2.1729],
        ...,
        [0.0000, 0.3190, 0.8542,  ..., 0.0000, 0.0000, 1.6953],
        [0.0000, 0.0305, 1.7972,  ..., 0.0000, 0.0000, 1.9260],
        [0.0000, 0

# 4. 总结

**1. 仅包含全连接层时，全连接层权值的初始化**

将权值 W 的标准差设置为 $\sqrt\frac{1}{n}$
```
def initialize(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight.data, std=np.sqrt(1/self.neural_nums))  # 权值初始化为正态分布，mean=0, std=根号1/n
```

<br/>

***

**2. 包含全连接层和饱和激活函数层时，全连接层权值的初始化：Xavier 权值初始化方法**

饱和激活函数：sigmoid 和 tanh 等

<br/>

若保持网络层输出值的方差在恰当范围，且同时考虑前向传播和反向传播，需要满足两个等式：

$\boldsymbol{n}_{\boldsymbol{i}} * \boldsymbol{D}(\boldsymbol{W})=\mathbf{1}$ （$n_i$ 为第 i 层神经元的数量）

$\boldsymbol{n}_{\boldsymbol{i+1}} * \boldsymbol{D}(\boldsymbol{W})=\mathbf{1}$

可得：$D(W)=\frac{2}{n_{i}+n_{i+1}}$。

<br/>

若使 Xavier 方法初始化的权值服从均匀分布 $U[-a, a]$，那么方差 $D(W)=\frac{(-a-a)^{2}}{12}=\frac{(2 a)^{2}}{12}=\frac{a^{2}}{3}$;

令$\frac{2}{n_{i}+n_{i+1}}=\frac{a^{2}}{3}$，解得：$\boldsymbol{a}=\frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}$；

<br/>

**所以 Xavier 方法将权值  $W$  初始化为均匀分布 $U\left[-\frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}, \frac{\sqrt{6}}{\sqrt{n_{i}+n_{i+1}}}\right]$ 时，可以保持网络层输出值的方差在恰当范围**（同时考虑了前向传播和反向传播）。
```
def initialize(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            a = np.sqrt(6 / (self.neural_nums + self.neural_nums))
            tanh_gain = nn.init.calculate_gain('tanh')
            a *= tanh_gain  # 上下限乘以 tanh 激活函数对输入值标准差的缩小比例
            nn.init.uniform_(m.weight.data, -a, a)  # 权值初始化为 [-a, a] 的均匀分布


def initialize(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            tanh_gain = nn.init.calculate_gain('tanh')
            nn.init.xavier_uniform_(m.weight.data, gain=tanh_gain)  # 直接调用 PyTorch 提供的 Xavier 初始化方法
```

<br/>

***

**3. 包含全连接层和非饱和激活函数层时，全连接层权值的初始化：Kaiming 权值初始化方法**

非饱和激活函数：relu 等

<br/>

若保持网络层的输出值在一个适当的范围，需要满足：（证明略）

$\mathrm{D}(W)=\frac{2}{n_{i}}$（针对 ReLU）

$\mathrm{D}(W)=\frac{2}{\left(1+a^{2}\right) n_{i}}$，$\operatorname{std}(W)=\sqrt{\frac{2}{\left(1+a^{2}\right) * n_{i}}}$（针对 ReLu 的变种，a 表示负半轴的斜率，$n_{i}$ 为输入层/上一层神经元的数量）

<br/>

**所以 Kaiming 方法将权值  $W$  初始化为正态分布 $N(0, std(w))$ 时，可以保持网络层输出值的方差在恰当范围。**
```
def initialize(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            nn.init.normal_(m.weight.data, std=np.sqrt(2 / self.neural_nums))  # 使用 Kaiming 方法初始化权值
                
def initialize(self):
    for m in self.modules():
        if isinstance(m, nn.Linear):
            nn.init.kaiming_normal_(m.weight.data)  # 直接调用 PyTorch 提供的 Kaiming 初始化方法
```

# 5. PyTorch 中提供的 10 中初始化方法

- Xavier 均匀分布
- Xavier 正态分布
- Kaiming 均匀分布
- Kaiming 正态分布
- 均匀分布
- 正态分布
- 常数分布
- 正交矩阵初始化
- 单位矩阵初始化
- 稀疏矩阵初始化

**权值初始化原则**：保持每一层输出值的方差一致，约为 1，即方差一致性原则；每一层的输出值不能太大，也不能太小。

# 6. nn.init.calculate_gain()

主要功能：计算激活函数的标准差变化尺度，即输入数据的标准差除以经过激活函数之后输出数据的标准差，即标准差的缩放比例。

在 Xavier 权值初始化方法中使用了 `tanh_gain = nn.init.calculate_gain('tanh')`

主要参数：
- nonlinearity：激活函数名称
- param：激活函数的参数，如 Leaky ReLU 的 negative_slop（负半轴斜率）。

代码实验：

In [8]:
import torch

x = torch.randn(10000) 
out = torch.tanh(x)

tanh_gain1 = x.std() / out.std() 
print('tanh_gain1:', tanh_gain1)

tanh_gain2 = nn.init.calculate_gain('tanh') 
print('tanh_gain2:', tanh_gain2)

tanh_gain1: tensor(1.5899)
tanh_gain2: 1.6666666666666667


结果显示：对于服从标准正态分布的输入值而言，tanh 激活函数会使输入值的标准差缩小 1.6 倍左右。